# Synthetic Shapes — Pairwise Distance Quality

Bifiltration: codensity + eccentricity (Alpha complex, lower-star).
$n = 1000$, noise $= 0$, 200 samples per class (1000 total).
CMD, MD 121, MD 10 for all pairs, separately for $H_0$ and $H_1$.

In [8]:
import numpy as np
import gudhi as gd
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances as pdist_sklearn
from scipy.stats import spearmanr
from tqdm import tqdm
import time, warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [9]:
n = 500
n_ex = 100
k_neighbors = 15

t_values = np.linspace(0, 1, 11)
a_fine = np.linspace(0.05, 0.95, 11)
b_fine = np.linspace(0, 1, 11)
a_coarse = np.array([0.25, 0.75])
b_coarse = np.linspace(0, 1, 5)
BOTTLENECK_E = 0.01

n_cmd = len(t_values)
n_md_fine = len(a_fine) * len(b_fine)
n_md_coarse = len(a_coarse) * len(b_coarse)
print(f'CMD: {n_cmd} | MD fine: {n_md_fine} | MD coarse: {n_md_coarse}')

CMD: 11 | MD fine: 121 | MD coarse: 10


In [10]:
def generate_circle(n, noise=0.0):
    theta = np.random.uniform(0, 2*np.pi, n)
    pts = np.column_stack([0.5+0.4*np.cos(theta), 0.5*np.ones(n), 0.5+0.4*np.sin(theta)])
    if noise > 0: pts += np.random.normal(0, noise, pts.shape)
    return pts
def generate_sphere(n, noise=0.0):
    phi = np.random.uniform(0, 2*np.pi, n)
    cos_theta = np.random.uniform(-1, 1, n)
    sin_theta = np.sqrt(1 - cos_theta**2)
    pts = np.column_stack([0.5+0.4*sin_theta*np.cos(phi), 0.5+0.4*sin_theta*np.sin(phi), 0.5+0.4*cos_theta])
    if noise > 0: pts += np.random.normal(0, noise, pts.shape)
    return pts
def generate_torus(n, noise=0.0):
    R, r = 0.3, 0.1
    theta = np.random.uniform(0, 2*np.pi, n)
    phi = np.random.uniform(0, 2*np.pi, n)
    pts = np.column_stack([0.5+(R+r*np.cos(phi))*np.cos(theta), 0.5+(R+r*np.cos(phi))*np.sin(theta), 0.5+r*np.sin(phi)])
    if noise > 0: pts += np.random.normal(0, noise, pts.shape)
    return pts
def generate_three_clusters(n, noise=0.0):
    centers = np.array([[0.25,0.25,0.5],[0.75,0.25,0.5],[0.5,0.75,0.5]])
    n_per = n // 3
    parts = [c + np.random.normal(0, 0.06, (n_per if i < 2 else n-2*n_per, 3)) for i, c in enumerate(centers)]
    pts = np.vstack(parts)
    if noise > 0: pts += np.random.normal(0, noise, pts.shape)
    return pts
def generate_two_circles(n, noise=0.0):
    n1 = n // 2; n2 = n - n1
    t1 = np.random.uniform(0, 2*np.pi, n1)
    t2 = np.random.uniform(0, 2*np.pi, n2)
    pts = np.vstack([
        np.column_stack([0.5+0.2*np.cos(t1), 0.5+0.2*np.sin(t1), 0.5*np.ones(n1)]),
        np.column_stack([0.5+0.4*np.cos(t2), 0.5+0.4*np.sin(t2), 0.5*np.ones(n2)])])
    if noise > 0: pts += np.random.normal(0, noise, pts.shape)
    return pts

generators = [generate_circle, generate_sphere, generate_torus, generate_three_clusters, generate_two_circles]
shape_names = ['Circle', 'Sphere', 'Torus', '3 Clusters', '2 Circles']
n_classes = len(generators)

In [11]:
Data, Labels = [], []
for cls_idx, gen in enumerate(generators):
    for _ in range(n_ex):
        Labels.append(cls_idx)
        Data.append(gen(n, noise=0.0))
Labels = np.array(Labels)
N = len(Data)
print(f'N = {N} ({n_classes} classes x {n_ex} samples)')

N = 500 (5 classes x 100 samples)


In [12]:
def compute_codensity(points, k):
    nn = NearestNeighbors(n_neighbors=k+1).fit(points)
    return nn.kneighbors(points)[0][:, -1]
def compute_eccentricity(points):
    return np.max(pdist_sklearn(points), axis=1)

class CachedSimplexTree:
    def __init__(self, points):
        self.st = gd.AlphaComplex(points=points).create_simplex_tree()
        self._simplices = [tuple(s) for s, _ in self.st.get_simplices()]
        max_dim = max(len(s) for s in self._simplices)
        self._vert_idx = np.full((len(self._simplices), max_dim), -1, dtype=np.int32)
        for i, s in enumerate(self._simplices):
            self._vert_idx[i, :len(s)] = s
    def compute_pd(self, func_values):
        f_ext = np.append(func_values, -np.inf)
        filt_vals = np.max(f_ext[self._vert_idx], axis=1)
        for i, s in enumerate(self._simplices):
            self.st.assign_filtration(s, float(filt_vals[i]))
        self.st.make_filtration_non_decreasing()
        self.st.persistence()
        pds = []
        for dim in range(2):
            pd = self.st.persistence_intervals_in_dimension(dim)
            if pd is not None and len(pd) > 0:
                pd = pd[np.isfinite(pd[:, 1])]
            if pd is None or len(pd) == 0:
                pd = np.empty((0, 2))
            pds.append(pd)
        return tuple(pds)

def phi_star_ab(phi1, phi2, a, b):
    return np.minimum(a, 1-a) * np.maximum((phi1-b)/a, (phi2+b)/(1-a))

def safe_bottleneck(pd1, pd2, e=0.01):
    if len(pd1) == 0 and len(pd2) == 0: return 0.0
    if len(pd1) == 0: return float(np.max((pd2[:,1]-pd2[:,0])/2))
    if len(pd2) == 0: return float(np.max((pd1[:,1]-pd1[:,0])/2))
    return gd.bottleneck_distance(pd1, pd2, e)

In [13]:
# Filtrations
Phi1, Phi2 = [], []
for data in Data:
    Phi1.append(compute_codensity(data, k_neighbors))
    Phi2.append(compute_eccentricity(data))
for phi_list in [Phi1, Phi2]:
    for i in range(len(phi_list)):
        f = phi_list[i]
        fmin, fmax = f.min(), f.max()
        if fmax > fmin:
            phi_list[i] = (f - fmin) / (fmax - fmin)

def make_cmd_filts(p1, p2): return [(1-t)*p1 + t*p2 for t in t_values]
def make_mdf_filts(p1, p2): return [phi_star_ab(p1, p2, a, b) for a in a_fine for b in b_fine]
def make_mdc_filts(p1, p2): return [phi_star_ab(p1, p2, a, b) for a in a_coarse for b in b_coarse]

params = [('cmd', make_cmd_filts, n_cmd),
          ('mdf', make_mdf_filts, n_md_fine),
          ('mdc', make_mdc_filts, n_md_coarse)]

PDs = {name: [None]*N for name, _, _ in params}
for i in tqdm(range(N), desc='PDs'):
    cst = CachedSimplexTree(Data[i])
    phi1, phi2 = Phi1[i], Phi2[i]
    for name, make_filts, _ in params:
        PDs[name][i] = [cst.compute_pd(f) for f in make_filts(phi1, phi2)]

PDs: 100%|██████████| 500/500 [08:14<00:00,  1.01it/s]


In [14]:
Ds = {}
for name, _, nparam in params:
    print(f'Distances: {name}')
    D_H0 = np.zeros((N, N))
    D_H1 = np.zeros((N, N))
    for i in tqdm(range(N), leave=False):
        for j in range(i+1, N):
            max_d0 = max_d1 = 0.0
            for p in range(nparam):
                d0 = safe_bottleneck(PDs[name][i][p][0], PDs[name][j][p][0], BOTTLENECK_E)
                d1 = safe_bottleneck(PDs[name][i][p][1], PDs[name][j][p][1], BOTTLENECK_E)
                if d0 > max_d0: max_d0 = d0
                if d1 > max_d1: max_d1 = d1
            D_H0[i,j] = D_H0[j,i] = max_d0
            D_H1[i,j] = D_H1[j,i] = max_d1
    Ds[name] = {'H0': D_H0, 'H1': D_H1}

Distances: cmd


Distances: mdf


Distances: mdc


In [15]:
np.save('Synthetic_distances.npy', {
    'd_cmd_H0': Ds['cmd']['H0'], 'd_cmd_H1': Ds['cmd']['H1'],
    'd_mdf_H0': Ds['mdf']['H0'], 'd_mdf_H1': Ds['mdf']['H1'],
    'd_mdc_H0': Ds['mdc']['H0'], 'd_mdc_H1': Ds['mdc']['H1'],
    'labels': Labels
})

In [16]:
triu = np.triu_indices(N, k=1)
for hdim in ['H0', 'H1']:
    d_cmd = Ds['cmd'][hdim][triu]
    d_mdf = Ds['mdf'][hdim][triu]
    d_mdc = Ds['mdc'][hdim][triu]
    sp_cmd_mdf, _ = spearmanr(d_mdf, d_cmd)
    sp_mdc_mdf, _ = spearmanr(d_mdf, d_mdc)
    sp_cmd_mdc, _ = spearmanr(d_cmd, d_mdc)
    print(f'\n{hdim}:')
    print(f'  Spearman CMD vs MD121: {sp_cmd_mdf:.4f}')
    print(f'  Spearman MD10 vs MD121: {sp_mdc_mdf:.4f}')
    print(f'  Spearman CMD vs MD10: {sp_cmd_mdc:.4f}')


H0:
  Spearman CMD vs MD121: 0.9973
  Spearman MD10 vs MD121: 0.9920
  Spearman CMD vs MD10: 0.9917

H1:
  Spearman CMD vs MD121: 0.9705
  Spearman MD10 vs MD121: 0.9721
  Spearman CMD vs MD10: 0.9876
